In [ ]:
# Install required dependencies
%pip install strands-agents>=0.1.2 strands-agents-tools>=0.1.1 python-dotenv


In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from mcp import StdioServerParameters, stdio_client
from strands import Agent
from strands.tools.mcp import MCPClient
from strands_tools import swarm, editor, file_write

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [2]:
# Get Perplexity API key from environment
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if not PERPLEXITY_API_KEY:
    print("⚠️ PERPLEXITY_API_KEY not found in environment variables.")
    print("Please set it in a .env file or uncomment the line below:")
    # PERPLEXITY_API_KEY = "your_api_key_here"  # Uncomment and add your key
else:
    print("✅ Perplexity API key loaded successfully!")


✅ Perplexity API key loaded successfully!


In [ ]:
def create_market_research_team():
    """
    Creates and returns the market research team with Perplexity integration.
    """
    # Set up Perplexity MCP server connection
    perplexity_mcp_server = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",
                args=[
                    "run",
                    "-i",
                    "--rm",
                    "-e",
                    "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask",
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY},
            )
        )
    )
    
    return perplexity_mcp_server

def conduct_market_research(project_description: str):
    """
    Conducts comprehensive market research for a given project.
    """
    try:
        perplexity_mcp_server = create_market_research_team()
        
        with perplexity_mcp_server:
            # Get available tools from Perplexity MCP server
            tools = perplexity_mcp_server.list_tools_sync()
            
            # Create Lead Market Analyst agent
            lead_market_analyst = Agent(
                system_prompt="""Conduct thorough analysis of the products and competitors, providing detailed 
                insights to guide marketing strategies. As the Lead Market Analyst at a digital marketing firm, you specialize 
                in understanding markets for new products and services. You can do web search to gather more information.
                """,
                tools=tools + [swarm],
            )

            # Create Chief Strategist agent
            chief_strategist = Agent(
                system_prompt="""Synthesize insights from market analysis to formulate marketing strategies. You are the Chief Strategist at a digital marketing agency,
                known for crafting custom marketing strategies that drive success of new products and services. You can do web search to gather more information.
                """,
                tools=tools,
            )

            print("\n🔍 Market Analyst is conducting research...\n")
            
            # Conduct market analysis using swarm of agents
            market_analyst_response = str(
                lead_market_analyst(
                    f"Use a swarm of 2 agents to conduct market analysis in competitive pattern. The project description is: \n\n{project_description}"
                )
            )

            print("\n💡 Chief Strategist is formulating strategy...\n")
            
            # Synthesize strategy from market analysis
            strategy_response = str(chief_strategist(market_analyst_response))

            return strategy_response

    except Exception as e:
        return f"❌ Error during market research: {str(e)}"

print("✅ Market research team functions defined!")


In [ ]:
def create_content_and_report(market_research: str):
    """
    Creates marketing content and formatted reports based on market research.
    """
    try:
        # Create Content Writer agent
        content_writer = Agent(
            system_prompt="""Develop and revise compelling and innovative content, including campaign ideas,
            campaign copy, and even detailed campaign reports. As a Content Writer at a digital marketing agency, you 
            excel in creating narratives that resonate with target audiences. 
            Your expertise is in turning marketing strategies into engaging stories and visual 
            content that captures attention and inspires action.
            """,
        )

        # Create Report Writer agent with file tools
        formatted_report_writer = Agent(
            system_prompt="""Produce and save a markdown-formatted report based on all of the outputs
            of your team's work. As a Report Writer you are excellent at taking everyone else's great work and 
            producing a cleanly formatted and easily readable report, saved in Markdown format 
            using file_write and editor tool. Create the file in current directory.
            """,
            tools=[file_write, editor],
        )

        print("\n✍️ Content Writer is creating marketing materials...\n")
        
        # Generate marketing content
        content_writer_response = str(content_writer(market_research))

        print("\n📄 Report Writer is formatting the final report...\n")
        
        # Create formatted report
        report_response = str(formatted_report_writer(content_writer_response))

        return report_response

    except Exception as e:
        return f"❌ Error during content creation: {str(e)}"

print("✅ Writer team functions defined!")


## Demo: Complete Startup Advisory Workflow

Now let's demonstrate the complete workflow with a sample startup project. We'll use the example of "FlyingCars" - a company that wants to be the leading supplier of flying cars.

### Step 1: Define the Project


In [ ]:
# Define the startup project for analysis
project_query = """
Create a marketing strategy for the following:

Project Name: FlyingCars wants to be the leading supplier of flying cars. 
The project is to build an innovative marketing strategy to showcase FlyingCars' advanced 
offerings, emphasizing ease of use, cost effectiveness, productivity, and safety. 
Target high net worth individuals, highlighting success stories and transformative 
potential. Be sure to include a draft for a video ad.
"""

print("🚀 Project defined:")
print(project_query)


### Step 2: Conduct Market Research

The market research team will analyze the flying car market, competitors, and target audience.


In [ ]:
# Run market research
print("🔍 Starting comprehensive market research...\n")

market_research_results = conduct_market_research(project_query)

print("\n" + "="*80)
print("📊 MARKET RESEARCH COMPLETED")
print("="*80)
print(market_research_results[:1000] + "..." if len(market_research_results) > 1000 else market_research_results)


### Step 3: Create Marketing Content and Reports

The writer team will transform the market research into compelling marketing materials and a comprehensive report.


In [ ]:
# Create content and reports
print("✍️ Creating marketing content and reports...\n")

final_report = create_content_and_report(market_research_results)

print("\n" + "="*80)
print("📋 FINAL REPORT COMPLETED")
print("="*80)
print(final_report[:1000] + "..." if len(final_report) > 1000 else final_report)


In [ ]:
# List generated files
import glob

markdown_files = glob.glob("*.md")
print("📁 Generated files:")
for file in markdown_files:
    if file != "README.md":  # Exclude the existing README
        print(f"  - {file}")
        
        # Show a preview of the file content
        try:
            with open(file, 'r') as f:
                content = f.read()
                print(f"\n📄 Preview of {file}:")
                print("-" * 40)
                print(content[:500] + "..." if len(content) > 500 else content)
                print("-" * 40)
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")


In [ ]:
# Customize this with your own startup idea
custom_project = """
Create a marketing strategy for the following:

Project Name: [Your Startup Name Here]
Description: [Describe your startup, its unique value proposition, target market, 
and what kind of marketing strategy you need]

Example:
Project Name: EcoClean - AI-powered sustainable cleaning service
Description: Build a marketing strategy for an AI-powered cleaning service that uses 
only eco-friendly products and optimizes routes for minimal environmental impact. 
Target environmentally conscious families and businesses. Emphasize sustainability, 
efficiency, and cost savings. Include social media campaign ideas.
"""

print("💡 Customize the project description above, then run the next cell!")


In [ ]:
# Run the complete workflow for your custom project
if "[Your Startup Name Here]" not in custom_project:
    print("🚀 Running analysis for your custom startup...\n")
    
    # Market research
    custom_research = conduct_market_research(custom_project)
    
    # Content creation
    custom_report = create_content_and_report(custom_research)
    
    print("\n✅ Analysis complete! Check the generated files above.")
else:
    print("⚠️ Please customize the project description in the previous cell first!")
